<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
    <h3 style="text-align: center; background-color:rgb(18, 135, 198); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>Table of Contents </b></h3>
    
- I. Crawling Data and Merging Datasets
- II. Exploring and Pre-processing Data
    - [2.1. Basic infomation](#2.1)
        - [2.1.1. Shape of data](#2.1.1)
        - [2.1.2. Meaning of each row](#2.1.2)
        - [2.1.3. Meaning of each column](#2.1.3)
        - [2.1.4. Check duplicate](#2.1.4)
        - [2.1.5. Data type of each column (standardize if necessary)](#2.1.5)
        - [2.1.6. Missing values of each column](#2.1.6)
    - [2.2 Data distribution](#2.2)
        - [2.2.1. Numerical columns](#2.2.1)
        - [2.2.2. Categorical columns](#2.2.2)
    - [2.3. Abnormal values and outliers](#2.3)
    - [2.4. Correlation between variables](#2.4)
- III. Asking meaningful questions
    - [3.1. Question 1](#3.1)
        - [3.1.1. Preprocessing](#3.1.1)
        - [3.1.2. Analysis](#3.1.2)
        - [3.1.3. Conclusion](#3.1.3)
    - [3.2. Question 2](#3.2)
        - [3.2.1. Preprocessing](#3.2.1)
        - [3.2.2. Analysis](#3.2.2)
        - [3.2.3. Conclusion](#3.2.3)
    - [3.3. Question 3](#3.3)
        - [3.3.1. Preprocessing](#3.3.1)
        - [3.3.2. Analysis](#3.3.2)
        - [3.3.3. Conclusion](#3.3.3)

    - [3.4. Question 4](#3.4)
        - [3.4.1. Preprocessing](#3.4.1)
        - [3.4.2. Analysis](#3.4.2)
        - [3.4.3. Conclusion](#3.4.3)
    - [3.5. Question 5](#3.5)
        - [3.5.1. Preprocessing](#3.5.1)
        - [3.5.2. Analysis](#3.5.2)
        - [3.5.3. Conclusion](#3.5.3)
    - [3.6. Question 6](#3.6)
        - [3.6.1. Preprocessing](#3.6.1)
        - [3.6.2. Analysis](#3.6.2)
        - [3.6.3. Conclusion](#3.6.3)
- IV. References

In [322]:
from IPython.display import display
import requests
import numpy as np
import pandas as pd
from typing import List
import os

import urllib.robotparser

<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
    <h3 style="text-align: center; background-color:rgb(2, 100, 153); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>1. Crawling Data and Merging Datasets </b></h3>

<a class="anchor" id="1.1."></a>
<h4 style="text-align: left; background-color:rgb(36, 138, 193); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>1.1. Helper functions </b></h4>

In [323]:
BASE_URL = 'http://api.worldbank.org/v2/'

INDICATOR_CODES = [
    "SP.POP.TOTL",
    "SP.POP.TOTL.FE.IN",
    "SP.POP.TOTL.MA.IN",
    "SP.DYN.CBRT.IN",
    "SP.DYN.CDRT.IN",
    "SE.COM.DURS",
    "SL.IND.EMPL.ZS",
    "SL.AGR.EMPL.ZS",
    "SL.SRV.EMPL.ZS",
    "SL.AGR.EMPL.FE.ZS",
    "SL.IND.EMPL.FE.ZS",
    "SL.SRV.EMPL.FE.ZS",
    "SL.UEM.TOTL.ZS",
    "SL.UEM.TOTL.FE.ZS",
    "NY.GDP.MKTP.CD",
    "NY.ADJ.NNTY.PC.KD.ZG",
    "NY.GSR.NFCY.CD",
    "NV.AGR.TOTL.CD",
    "NV.AGR.TOTL.ZS",
    "NV.IND.TOTL.ZS",
    "NV.SRV.TOTL.ZS",
    "GC.XPN.TOTL.GD.ZS",
    "NY.GDP.PCAP.CD",
    "NY.GDP.MKTP.KD.ZG",
    "BX.KLT.DINV.CD.WD",
    "NE.EXP.GNFS.ZS",
    "NE.IMP.GNFS.ZS",
    "SP.DYN.LE00.MA.IN",
    "SP.DYN.LE00.FE.IN",
    "FP.CPI.TOTL.ZG",
]

features_mapping = {
    "SP.POP.TOTL": "Total Population",
    "SP.POP.TOTL.FE.IN": "Female Population",
    "SP.POP.TOTL.MA.IN": "Male Population",
    "SP.DYN.CBRT.IN": "Birth Rate",
    "SP.DYN.CDRT.IN": "Death Rate",
    "SE.COM.DURS": "Compulsory Education Dur.",
    "SL.IND.EMPL.ZS": "Employment in Industry(%)",
    "SL.AGR.EMPL.ZS": "Employment in Agriculture(%)",
    "SL.SRV.EMPL.ZS": "Employment in Services(%)",
    "SL.AGR.EMPL.FE.ZS": "Female Employment in Agriculture(%)",
    "SL.IND.EMPL.FE.ZS": "Female Employment in Industry(%)",
    "SL.SRV.EMPL.FE.ZS" : "Female Employment in Services(%)",
    "SL.UEM.TOTL.ZS": "Unemployment(%)",
    "SL.UEM.TOTL.FE.ZS" : "Female Unemployment(%)",
    "NY.GDP.MKTP.CD": "GDP in USD",
    "NY.ADJ.NNTY.PC.KD.ZG": "National Income per Capita",
    "NY.GSR.NFCY.CD": "Net income from Abroad",
    "NV.AGR.TOTL.CD": "Agriculture value added(in USD)",
    "NV.AGR.TOTL.ZS" : "Agriculture value added(%)",
    "NV.IND.TOTL.ZS" : "Industry value added(%)",
    "NV.SRV.TOTL.ZS" : "Services value added(%)",
    "GC.XPN.TOTL.GD.ZS" : "Expense(% of GDP)",
    "NY.GDP.PCAP.CD": "GDP per capita",
    "NY.GDP.MKTP.KD.ZG": "GDP growth rate",
    "BX.KLT.DINV.CD.WD": "Foreign Direct Investment",
    "NE.EXP.GNFS.ZS": "Exports of goods and services",
    "NE.IMP.GNFS.ZS": "Imports of goods and services",
    "SP.DYN.LE00.MA.IN": "Male life expectancy",
    "SP.DYN.LE00.FE.IN": "Female life expectancy",
    "FP.CPI.TOTL.ZG" : "Inflation rate",
}

In [324]:
def loadData(country_code: str, format: str = "json", per_page: int = 100, year_interval: str = '2015:2022') -> List:
    result_data = []
    
    year_start = int(year_interval.split(":")[0])
    year_end = int(year_interval.split(":")[1])
    
    for indicator in INDICATOR_CODES:
        indicators = []
        url = f"{BASE_URL}country/{country_code}/indicator/{indicator}?format=json&per_page={per_page}&date={year_interval}"
        response = requests.get(url)
        print(url)
        
        if response.status_code == 200 and "message" not in response.json()[0].keys():
            data = response.json()[1]
            for item in data:
                value = item['value']
                indicators.append(float(value) if value is not None else np.nan)
        else:
            indicators.extend([np.nan] * (year_end - year_start + 1))  # Extend with NaNs if no data
    
        result_data.append(indicators)
    
    # Add the year range in descending order as integers
    result_data.append(list(range(year_end, year_start - 1, -1)))

    return result_data

In [325]:
def loadDataCountry(country_code: str, format: str = "json", per_page: int = 100, year_interval: str = '2015:2022', is_display: bool = False) -> pd.DataFrame:
    df_country_code = pd.read_csv("data/all_countries.csv")
    df_country_code = df_country_code.fillna("NA")
    COUNTRIES_MAPPING = dict(zip(df_country_code["iso2Code"], df_country_code["name"]))
    del df_country_code

    col_list = list(features_mapping.values()) + ['Year']
    print(f"Loading data for {COUNTRIES_MAPPING[country_code]}")

    dataLst = loadData(country_code=country_code, format=format, per_page=per_page, year_interval=year_interval)

    # Ensure all lists have the same length
    max_len = max(len(arr) for arr in dataLst)
    dataLst_padded = [
        np.pad(
            np.array([x if x is not None else np.nan for x in arr], dtype=float),
            (0, max_len - len(arr)),
            constant_values=np.nan
        )
        for arr in dataLst
    ]

    df = pd.DataFrame(np.column_stack(dataLst), columns=col_list)
    df['Country'] = COUNTRIES_MAPPING[country_code]

    if is_display:
        display(df.head())

    return df

In [326]:
def replacer(s, newstring, index, nofail=False):
    if not nofail and index not in range(len(s)):
        raise ValueError("index outside given string")
    if index < 0:  # add it to the beginning
        return newstring + s
    if index > len(s):  # add it to the end
        return s + newstring
    return s[:index] + newstring + s[index + 1:]

In [327]:
def loadDataContinent(format: str = "json", per_page: int = 100, year_interval: str = '2015:2022', is_display: bool = False) -> pd.DataFrame:
    df_country_code = pd.read_csv("data/asian_countries.csv")
    lst_country_codes = df_country_code["iso2Code"].to_list()

    df = pd.DataFrame()
    for country_code in lst_country_codes:
        df = pd.concat([df, loadDataCountry(
            country_code=country_code,
            format=format,
            per_page=per_page,
            year_interval=year_interval,
            is_display=False)], axis=0)

    if is_display:
        display(df.head())

    return df

In [328]:
def saveDataFrame2CSV(df: pd.DataFrame, save_path: str, sep: str = ',', encoding: str = 'utf-8') -> bool:
    try:
        df.to_csv(save_path, sep=sep, encoding=encoding, index=False)
    except:
        raise ModuleNotFoundError
        # return False
    return True

In [329]:
years = '2000:2023'

- First we get all countries

In [330]:
url_countries = f"{BASE_URL}country?format=json&per_page=500"
response = requests.get(url_countries)
countries = []
if response.status_code == 200:
    data = response.json()
    if len(data) > 1 and isinstance(data[1], list):  # Check if data exists
        countries = [
            {
                "name": country["name"],
                "iso2Code": country["iso2Code"],
                "iso3Code": country["id"],
                "region": country["region"]["value"]
            }
            for country in data[1]
        ]
        print(f"Fetched {len(countries)} countries.")
    else:
        print("No country data found.")
else:
    print(f"Error fetching data: {response.status_code}")

print(countries[:5])


Fetched 296 countries.
[{'name': 'Aruba', 'iso2Code': 'AW', 'iso3Code': 'ABW', 'region': 'Latin America & Caribbean '}, {'name': 'Africa Eastern and Southern', 'iso2Code': 'ZH', 'iso3Code': 'AFE', 'region': 'Aggregates'}, {'name': 'Afghanistan', 'iso2Code': 'AF', 'iso3Code': 'AFG', 'region': 'South Asia'}, {'name': 'Africa', 'iso2Code': 'A9', 'iso3Code': 'AFR', 'region': 'Aggregates'}, {'name': 'Africa Western and Central', 'iso2Code': 'ZI', 'iso3Code': 'AFW', 'region': 'Aggregates'}]


In [331]:
countries_df = pd.DataFrame(countries)
countries_df.to_csv("data/all_countries.csv", index=False)

- Second get all asian countries

In [332]:
countries_df = pd.read_csv("data/all_countries.csv")

asian_countries = countries_df[countries_df["region"].str.contains("Asia")]
asian_countries.to_csv("data/asian_countries.csv", index=False)


In [333]:
# df_jp = loadDataCountry("JP", year_interval=years, is_display=True)
# df_cn = loadDataCountry("CN", year_interval=years, is_display=True)
# df_kr = loadDataCountry("KR", year_interval=years, is_display=True)

In [334]:
df = loadDataContinent(year_interval=years, is_display=True)
df

Loading data for Afghanistan
http://api.worldbank.org/v2/country/AF/indicator/SP.POP.TOTL?format=json&per_page=100&date=2000:2023
http://api.worldbank.org/v2/country/AF/indicator/SP.POP.TOTL.FE.IN?format=json&per_page=100&date=2000:2023
http://api.worldbank.org/v2/country/AF/indicator/SP.POP.TOTL.MA.IN?format=json&per_page=100&date=2000:2023
http://api.worldbank.org/v2/country/AF/indicator/SP.DYN.CBRT.IN?format=json&per_page=100&date=2000:2023
http://api.worldbank.org/v2/country/AF/indicator/SP.DYN.CDRT.IN?format=json&per_page=100&date=2000:2023
http://api.worldbank.org/v2/country/AF/indicator/SE.COM.DURS?format=json&per_page=100&date=2000:2023
http://api.worldbank.org/v2/country/AF/indicator/SL.IND.EMPL.ZS?format=json&per_page=100&date=2000:2023
http://api.worldbank.org/v2/country/AF/indicator/SL.AGR.EMPL.ZS?format=json&per_page=100&date=2000:2023
http://api.worldbank.org/v2/country/AF/indicator/SL.SRV.EMPL.ZS?format=json&per_page=100&date=2000:2023
http://api.worldbank.org/v2/country

,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Compulsory Education Dur.,Employment in Industry(%),Employment in Agriculture(%),Employment in Services(%),Female Employment in Agriculture(%),...,GDP per capita,GDP growth rate,Foreign Direct Investment,Exports of goods and services,Imports of goods and services,Male life expectancy,Female life expectancy,Inflation rate,Year,Country
0,41454761.0,20530683.0,20924078.0,NaN,NaN,9.0,18.386784,45.463952,36.149276,47.417946,...,415.707417,2.710887,NaN,16.775136,50.498207,NaN,NaN,NaN,2023.0,Afghanistan
1,40578842.0,20103167.0,20475675.0,35.143,6.906,9.0,18.540741,45.592654,35.866618,47.846236,...,357.261153,-6.240172,NaN,18.380042,54.505427,59.766,66.213,NaN,2022.0,Afghanistan
2,40000412.0,19813956.0,20186456.0,35.842,7.344,9.0,18.522262,45.955199,35.522526,53.755655,...,356.496214,-20.738839,3.079096e+07,14.342153,37.069564,58.915,65.279,NaN,2021.0,Afghanistan
3,39068979.0,19343529.0,19725450.0,36.051,7.113,9.0,18.481312,45.983411,35.535277,53.835873,...,510.787063,-2.351101,1.297015e+07,10.420817,36.289077,59.866,65.432,NaN,2020.0,Afghanistan
4,37856121.0,18736474.0,19119647.0,36.466,6.791,9.0,18.214766,45.050959,36.734276,58.510279,...,496.602504,3.911603,2.340455e+07,NaN,NaN,60.619,66.677,2.302373,2019.0,Afghanistan


,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Compulsory Education Dur.,Employment in Industry(%),Employment in Agriculture(%),Employment in Services(%),Female Employment in Agriculture(%),...,GDP per capita,GDP growth rate,Foreign Direct Investment,Exports of goods and services,Imports of goods and services,Male life expectancy,Female life expectancy,Inflation rate,Year,Country
0,41454761.0,20530683.0,20924078.0,NaN,NaN,9.0,18.386784,45.463952,36.149276,47.417946,...,415.707417,2.710887,NaN,16.775136,50.498207,NaN,NaN,NaN,2023.0,Afghanistan
1,40578842.0,20103167.0,20475675.0,35.143,6.906,9.0,18.540741,45.592654,35.866618,47.846236,...,357.261153,-6.240172,NaN,18.380042,54.505427,59.766,66.213,NaN,2022.0,Afghanistan
2,40000412.0,19813956.0,20186456.0,35.842,7.344,9.0,18.522262,45.955199,35.522526,53.755655,...,356.496214,-20.738839,3.079096e+07,14.342153,37.069564,58.915,65.279,NaN,2021.0,Afghanistan
3,39068979.0,19343529.0,19725450.0,36.051,7.113,9.0,18.481312,45.983411,35.535277,53.835873,...,510.787063,-2.351101,1.297015e+07,10.420817,36.289077,59.866,65.432,NaN,2020.0,Afghanistan
4,37856121.0,18736474.0,19119647.0,36.466,6.791,9.0,18.214766,45.050959,36.734276,58.510279,...,496.602504,3.911603,2.340455e+07,NaN,NaN,60.619,66.677,2.302373,2019.0,Afghanistan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,1822773.0,910228.0,912545.0,17.781,5.032,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.338531e+07,NaN,NaN,73.663,76.962,-1.064727,2004.0,Kosovo
20,1826434.0,909898.0,916536.0,15.749,5.021,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,73.311,76.639,-1.084813,2003.0,Kosovo
21,1829462.0,909376.0,920086.0,16.858,5.014,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,73.045,76.312,NaN,2002.0,Kosovo
22,1820009.0,903646.0,916363.0,17.249,4.992,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,72.689,76.109,NaN,2001.0,Kosovo


In [335]:
df.to_csv("data/asian_countries_data.csv", index=False)

In [339]:
raw_df = pd.read_csv("data/asian_countries_data.csv")
def missing_ratio(col):
    return (col.isna().sum() / len(col) * 100).round(1)

def lower_quartile(col):
    return col.quantile(0.25).round(1)

def median(col):
    return col.median().round(1)

def upper_quartile(col):
    return col.quantile(0.75).round(1)

num_cols = raw_df.select_dtypes(include=['number']).columns
num_col_info_df = pd.DataFrame(index=['missing_ratio', 'min', 'lower_quartile', 'median', 'upper_quartile', 'max'])

for column in num_cols:

    #Calculate descriptive statistic for each column then store it into dataframe
    missing_rate = (raw_df[column].isnull().sum() / len(raw_df[column]) * 100).round(1)
    min_val = raw_df[column].min().round(1)
    max_val = raw_df[column].max().round(1)
    median_val = raw_df[column].median().round(1)
    lower_quart = raw_df[column].quantile(0.25).round(1)
    upper_quart = raw_df[column].quantile(0.75).round(1)

    num_col_info_df[column] = [missing_rate, min_val, lower_quart, median_val, upper_quart, max_val]


dict(num_col_info_df.iloc[0])


{'Total Population': 0.0,
 'Female Population': 0.0,
 'Male Population': 0.0,
 'Birth Rate': 6.9,
 'Death Rate': 8.1,
 'Compulsory Education Dur.': 18.7,
 'Employment in Industry(%)': 17.6,
 'Employment in Agriculture(%)': 16.6,
 'Employment in Services(%)': 16.6,
 'Female Employment in Agriculture(%)': 17.6,
 'Female Employment in Industry(%)': 16.6,
 'Female Employment in Services(%)': 16.6,
 'Unemployment(%)': 16.6,
 'Female Unemployment(%)': 16.6,
 'GDP in USD': 3.1,
 'National Income per Capita': 43.5,
 'Net income from Abroad': 14.4,
 'Agriculture value added(in USD)': 13.0,
 'Agriculture value added(%)': 13.0,
 'Industry value added(%)': 11.5,
 'Services value added(%)': 12.9,
 'Expense(% of GDP)': 37.5,
 'GDP per capita': 3.1,
 'GDP growth rate': 7.2,
 'Foreign Direct Investment': 17.7,
 'Exports of goods and services': 14.2,
 'Imports of goods and services': 14.2,
 'Male life expectancy': 9.7,
 'Female life expectancy': 9.7,
 'Inflation rate': 20.1,
 'Year': 0.0}